## Este notebook va a ser exclusivamente para analizar qué modelo de ML es más óptimo para nuestro dataset, generarlo, entrenarlo y guardarlo para usarlo en streamlit.

In [55]:
# Importamos las librerías
import pandas as pd
import numpy as np
from pycaret.regression import *
from fast_ml import eda
import seaborn as sns
import matplotlib.pyplot as plt

In [56]:
# Cargamos el dataset
valencia = pd.read_csv("data/valencia_barrio_municipio.csv")
valencia.drop('Municipio', axis =1, inplace=True)
valencia.drop('Unnamed: 0', axis=1, inplace=True)
valencia.drop('ID', axis=1, inplace=True)
valencia

,Precio,Preciom2,Metros_Construidos,Habitaciones,Baños,Terraza,Ascensor,Aire_Acondicionado,Servicios,Garaje,...,Año_diseñado,Plantas_máximas,Número_viviendas,Calidad_catastral,Distancia_centro,Distancia_metro,Distancia_Blasco,Latitud,Longitud,Barrio
0,111000.0,1480.00,75,2,1,1,1,1,2,1,...,2001,7,35,6,3.759093,0.756517,3.373132,39.500908,-0.393538,CIUTAT FALLERA
1,169000.0,1320.31,128,4,2,0,1,1,3,1,...,1976,7,56,5,2.073773,0.370680,0.540594,39.482599,-0.359125,BENIMACLET
2,162000.0,1883.72,86,2,1,0,1,1,3,0,...,2010,6,20,4,2.099641,0.038560,0.719203,39.485144,-0.362470,BENIMACLET
3,199000.0,1792.79,111,4,2,0,1,0,3,0,...,1977,8,25,5,2.120917,0.524827,0.915379,39.487445,-0.367417,BENIMACLET
4,73000.0,1303.57,56,3,1,1,1,0,3,0,...,1960,6,20,8,3.087950,1.129985,2.009535,39.497057,-0.370182,ELS ORRIOLS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33617,121000.0,1008.33,120,3,2,0,1,1,3,0,...,1982,4,13,5,2.800362,0.710564,2.607601,39.492210,-0.391340,BENICALAP
33618,136000.0,1619.05,84,2,1,1,1,1,3,0,...,1970,6,49,7,2.282765,0.321158,2.404973,39.486174,-0.392554,EL CALVARI
33619,98000.0,867.26,113,2,2,1,0,0,3,0,...,1974,6,17,7,2.583955,0.661318,2.510628,39.489754,-0.391909,BENICALAP
33620,45000.0,661.76,68,3,1,0,0,0,3,0,...,1963,6,41,7,3.420772,0.529830,3.305286,39.495606,-0.398167,BENICALAP


In [53]:
# Importamos LabelEncoder.
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

# Lo aplicamos a nuestra columna sin encodear.
valencia['Barrio'] = le.fit_transform(valencia['Barrio'])

# Mostramos para comprobarlo un head.
valencia.head(8)


,Precio,Preciom2,Metros_Construidos,Habitaciones,Baños,Terraza,Ascensor,Aire_Acondicionado,Servicios,Garaje,...,Año_diseñado,Plantas_máximas,Número_viviendas,Calidad_catastral,Distancia_centro,Distancia_metro,Distancia_Blasco,Latitud,Longitud,Barrio
0,111000.0,1480.00,75,2,1,1,1,1,2,1,...,2001,7,35,6,3.759093,0.756517,3.373132,39.500908,-0.393538,14
1,169000.0,1320.31,128,4,2,0,1,1,3,1,...,1976,7,56,5,2.073773,0.370680,0.540594,39.482599,-0.359125,5
2,162000.0,1883.72,86,2,1,0,1,1,3,0,...,2010,6,20,4,2.099641,0.038560,0.719203,39.485144,-0.362470,5
3,199000.0,1792.79,111,4,2,0,1,0,3,0,...,1977,8,25,5,2.120917,0.524827,0.915379,39.487445,-0.367417,5
4,73000.0,1303.57,56,3,1,1,1,0,3,0,...,1960,6,20,8,3.087950,1.129985,2.009535,39.497057,-0.370182,24
5,45000.0,692.31,65,3,1,0,0,0,3,0,...,1963,6,174,8,2.723051,0.950909,1.624347,39.493621,-0.369603,24
6,81000.0,1012.50,80,3,1,0,1,0,3,0,...,1964,6,47,7,2.967437,0.936006,1.837951,39.495687,-0.368560,24
7,85000.0,923.91,92,4,1,0,0,0,3,0,...,1978,5,21,7,2.744555,1.080070,1.697090,39.494040,-0.371029,24


Únicamente tenemos una columna que debemos encodear, ya que la hemos añadido recientemente. El resto del dataframe está encodeado previamente en el preprocesamiento del mismo. Vamos a usar la librería sklearn para encodear dicha columna.

In [ ]:
def replace_outliers(df, f=1.5):
    # Select only numeric columns
    numeric_cols = df.select_dtypes(include='number').columns
    
    Q1 = df[numeric_cols].quantile(0.25)
    Q3 = df[numeric_cols].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - f * IQR
    upper_bound = Q3 + f * IQR
    
    for col in numeric_cols:
        df.loc[df[col] < lower_bound[col], col] = lower_bound[col]
        df.loc[df[col] > upper_bound[col], col] = upper_bound[col]
    
    return df
replace_outliers(valencia)

In [ ]:
# Usando librería de fastml vemos como queda el dataframe.
eda.df_info(valencia)

In [ ]:
# Creamos nuestro setup con la columna objetivo
setup = setup(valencia, target='Precio', session_id=323)

* Session id: Identificador único de la sesión, que se utiliza para identificar y mantener la consistencia entre los diferentes pasos del proceso.
* Target: Variable objetivo de la predicción, que en este caso es "Precio".
* Target type: Tipo de variable objetivo, que en este caso es "Regression" (regresión).
* Original data shape: Forma original del conjunto de datos antes de cualquier transformación, que tiene una longitud de 33,622 filas y 36 columnas.
* Transformed data shape: Forma del conjunto de datos después de la transformación, que tiene la misma longitud y columnas que el conjunto de datos original.
* Transformed train set shape: Forma del conjunto de entrenamiento después de la transformación, que tiene 23,535 filas y 36 columnas.
* Transformed test set shape: Forma del conjunto de prueba después de la transformación, que tiene 10,087 filas y 36 columnas.
* Numeric features: Número de características numéricas en el conjunto de datos, que es de 35.
* Preprocess: Indica si se ha preprocesado o no el conjunto de datos.
* Imputation type: Tipo de imputación de valores faltantes, que en este caso es "simple".
* Numeric imputation: Método utilizado para imputar los valores faltantes en las características numéricas, que en este caso es el promedio (mean).
* Categorical imputation: Método utilizado para imputar los valores faltantes en las características categóricas, que en este caso es la moda (mode).
* Fold Generator: Método utilizado para generar los pliegues en la validación cruzada, que en este caso es "KFold".
* Fold Number: Número de pliegues utilizados en la validación cruzada, que en este caso es 10.
* CPU Jobs: Número de núcleos de CPU utilizados para el entrenamiento, que en este caso es -1 (todos los núcleos disponibles).
* Use GPU: Indica si se ha utilizado o no una GPU para el entrenamiento, que en este caso es "False".
* Log Experiment: Indica si se ha registrado o no el experimento, que en este caso es "False".
* Experiment Name: Nombre del experimento, que en este caso es "reg-default-name".
* USI: Identificador único del experimento, que se utiliza para identificar y mantener la consistencia entre los diferentes pasos del proceso.

In [ ]:
# Evaluamos los modelos
best = compare_models()

In [ ]:
print(best)

In [ ]:
et = create_model("et")

In [ ]:
tune_et = tune_model(et)

In [ ]:
# Mostramos gráficamente la matriz de confusión.
plot_model(et)

En mi caso, un R^2 de 1 en el conjunto de entrenamiento significa que el modelo puede explicar perfectamente la varianza en los datos de entrenamiento, lo cual puede indicar que el modelo está sobreajustado a los datos de entrenamiento y puede no ser tan bueno en datos nuevos.

Por otro lado, un R^2 de 0.991 en el conjunto de prueba indica que el modelo puede explicar la varianza en los datos de prueba en un 99,1%, lo cual sugiere que el modelo es bueno en generalización y puede ser útil para hacer predicciones en nuevos datos.

In [ ]:
# Predición modelo.
holdout_pred = predict_model(et)

* MAE (Error Absoluto Medio): Este valor representa la media de las desviaciones absolutas entre los valores reales y las predicciones del modelo. En mi caso, el valor obtenido es de 2137.4601. Podemos considerar este valor como relativamente bajo, lo que significa que el modelo tiene un buen rendimiento en términos de precisión.
* MSE (Error Cuadrático Medio): Este valor representa la media de las desviaciones al cuadrado entre los valores reales y las predicciones del modelo. En mi caso, el valor obtenido es de 306132376.6829. Este valor es relativamente alto, lo que puede indicar que el modelo tiene un sesgo. Sin embargo, dado que el modelo ha obtenido un buen rendimiento en términos de MAE, podemos considerar que el sesgo no es muy significativo.
* RMSE (Raíz del Error Cuadrático Medio): Este valor representa la raíz cuadrada del valor de MSE. En mi caso, el valor obtenido es de 17496.6390. Este valor nos indica que el modelo tiene un margen de error de alrededor de 17500 unidades al predecir el precio de una vivienda. Este valor puede ser considerado como relativamente bajo en comparación con los valores de los precios de las viviendas, lo que sugiere que el modelo tiene un buen rendimiento en términos de precisión.
* R2 (Coeficiente de Determinación): Este valor representa la proporción de la varianza en los valores de respuesta que es explicada por el modelo. En mi caso, el valor obtenido es de 0.9908, lo que significa que el modelo explica el 99.08% de la varianza en los precios de las viviendas. Este valor es relativamente alto, lo que sugiere que el modelo tiene un buen rendimiento en términos de precisión.
* RMSLE (Raíz del Error Cuadrático Medio del Logaritmo): Este valor es similar al RMSE, pero se aplica a los valores de los logaritmos de las predicciones y los valores reales. En mi caso, el valor obtenido es de 0.0184. Este valor es relativamente bajo, lo que sugiere que el modelo tiene un buen rendimiento en términos de precisión.
* MAPE (Error Porcentual Absoluto Medio): Este valor representa la media de las desviaciones porcentuales absolutas entre los valores reales y las predicciones del modelo. En mi caso, el valor obtenido es de 0.0066, lo que significa que el modelo tiene un margen de error de alrededor del 0.66%. Este valor puede considerarse como relativamente bajo, lo que sugiere que el modelo tiene un buen rendimiento en términos de precisión.

### Voy a proceder a crear, entrenar y guardar el modelo por mi cuenta, en vez de usar pycaret ya que el RMSE que me aparece es relativamente alto.

In [ ]:
# Importamos las librerías necesarias
from sklearn.ensemble import ExtraTreesRegressor
import joblib

# Creamos el modelo
model = ExtraTreesRegressor(n_estimators=100, max_depth=10, random_state=42)

# Entrenamos el modelo con nuestros datos de entrenamiento
model.fit(X_train, y_train)

# Hacemos predicciones con el conjunto de datos de prueba
y_pred = model.predict(X_test)

# Evaluamos el modelo
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

# Guardamos el modelo entrenado en un archivo
joblib.dump(model, 'extratreesregressor_model.joblib')
